In [ ]:
import os
import time
import ollama
import pandas as pd
import tqdm

FRENCH = True

In [ ]:
def messages_from_prompt(prompt):
    messages = []
    with open(f"prompts/{prompt}", "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    for line in lines:
        role = line.split(" ")[0]
        content = line[len(role)+1:]
        messages.append({"role": role, "content": content})
    return messages

In [ ]:
# for all prompts evaluate the model

# all the text files in the prompts directory
prompts = [f for f in os.listdir("prompts") if f.endswith(".txt")]
# models = ["gemma", "llama2", "mistral", "openbuddy-mistral", "openchat", "openhermes", "qwen:4b", "qwen:7b", "vigogne", "vigostral"]
models = ["openbuddy-mistral", "vigogne", "vigostral"]

display = False
display_minimal = True

for round_number in range(500):

    if display or display_minimal:print("Starting round", round_number)

    time_str = time.strftime("%Y-%m-%d_%H-%M-%S")
    time_df = pd.DataFrame(columns=["model", "prompt", "time_per_char", "time_init", "time_total"])

    # new tqdm progress bar
    pbar = tqdm.tqdm(total=len(models) * len(prompts))

    for model in models:
        if display:print(f"  Model: {model}")
        ollama.chat(
            model=model,
            messages=[],
            stream=False
        )
        for prompt in prompts:
            if display:print(f"Prompt: {prompt}")
            messages = messages_from_prompt(prompt)

            response = ""
            first_chunk = True
            start = time.time()
            stream = ollama.chat(
                model=model,
                messages=messages,
                stream=True
            )
            for chunk in stream:
                if first_chunk:
                    before_stream = time.time()
                    first_chunk = False
                response += chunk['message']['content']
                if display:print(response, end="\r")
            end = time.time()

            new_df = pd.DataFrame([{
                "model": model,
                "prompt": prompt,
                "time_per_char": (end - before_stream) / len(response) if len(response) > 0 else end - before_stream,
                "time_init": before_stream - start,
                "time_total": end - start
            }])
            if len(time_df) == 0:
                time_df = new_df.copy()
            else:
                time_df = pd.concat([time_df, new_df])
            if display:print(f"Time: {end - start}")
            model_str = model.replace(":", "")
            with open(f"responses/{model_str}_chat_{time_str}_{prompt}", "w", encoding="utf-8") as f:
                f.write(response)
            
            # update the progress bar
            pbar.update(1)

            time.sleep(1)

    # write the time df to a file
    with open(f"time_dfs/time_df_{time_str}.csv", "w") as f:
        time_df.to_csv(f, index=False)